In [ ]:
import tensorflow as tf
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from keras import Model, layers
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

2.6.0


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    device = torch.device("cuda")
    print('GPU:', torch.cuda.get_device_name(0))
else:
    raise SystemError('GPU device not found')

GPU: Tesla K80


In [ ]:
try:
    drive.mount('/content/drive')
except:
    print("You are not working in Colab at the moment :(")

Mounted at /content/drive


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

In [ ]:
EPOCHS = 30
CLASSES = 2
BATCH_SIZE = 8
IMG_WIDTH = 240
IMG_HEIGHT = 240

In [ ]:

TRAIN_PATH = '/tmp/train'
VAL_PATH = '/tmp/val'
TEST_PATH = '/tmp/test'

total_train_data = len(list(paths.list_images(TRAIN_PATH)))
total_val_data = len(list(paths.list_images(VAL_PATH)))
total_test_data = len(list(paths.list_images(TEST_PATH)))

train_datagen = ImageDataGenerator(rescale=1 / 255.0,
                                   rotation_range=360,
                                   width_shift_range=[-10,10],
                                   height_shift_range=[-10,10],
                                   shear_range=0.2,
                                   #zoom_range=0.2,
                                   brightness_range=[1.0, 5.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

train_data = train_datagen.flow_from_directory(
    TRAIN_PATH,
    class_mode="binary",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE)


val_datagen = ImageDataGenerator(rescale=1 / 255.0)
validation_data = val_datagen.flow_from_directory(
    VAL_PATH,
    class_mode="binary",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data = test_datagen.flow_from_directory(
    TEST_PATH,
    class_mode="binary",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)

train_data.class_indices

Found 480 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


{'flat.class': 0, 'full.class': 1}

In [ ]:
conv_base = tf.keras.applications.VGG19(
    include_top=False,
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
    weights='imagenet')

for layer in conv_base.layers[:-1]:
    layer.trainable = False

x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(64, activation='relu')(x) 
predictions = layers.Dense(CLASSES, activation='sigmoid')(x)
model = Model(conv_base.input, predictions)

model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy', 'mse'])
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=EPOCHS)



predIdxs = model.predict(test_data)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(test_data.classes, predIdxs, target_names=test_data.class_indices.keys()))
print(confusion_matrix(test_data.classes, predIdxs))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 240, 240, 16)      208       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 120, 120, 16)      0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 120, 120, 16)      1040      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 60, 60, 16)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 60, 60, 32)        2080      
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 30, 30, 32)      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"

font = {'size': 12.5}

plt.rc('font', **font)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, EPOCHS + 1)

plt.figure(figsize=(8, 8))

plt.subplot(2, 1, 1)
plt.plot(epochs, acc, label='Training Accuracy', color = 'green')
plt.plot(epochs, val_acc, label='Validation Accuracy', color = 'orange')
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
plt.legend(loc='lower right')
plt.ylabel('Accuracy', fontsize=16)
plt.ylim([min(plt.ylim()),1])
plt.title('ResNet-50 Training and Validation Accuracy', pad=10,fontweight="bold")
plt.xlabel('Epoch', fontsize=16)
plt.tight_layout()

plt.subplot(2, 1, 2)  
plt.plot(epochs, loss, label='Training Loss', color = 'red')
plt.plot(epochs, val_loss, label='Validation Loss', color = 'blue')
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy', fontsize=16)
plt.ylim([0,1.0])
plt.title('ResNet-50 Training and Validation Loss', pad=10, fontweight="bold")
plt.xlabel('Epoch', fontsize=16)
plt.tight_layout()

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.5)
                    
plt.show()